# Dynamic Programming Corner Growth

In [1]:
# Initialize grid and set up probability density
# Run random calls

In [2]:
using Random, Distributions, Plots, Interact

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-1200088582239717073\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-4989605791935985712\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-1200088582239717073\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
p = .5
mat_size = 20
indep_draws = rand(Geometric(p), mat_size, mat_size)

20×20 Array{Int64,2}:
 2  1  0  1  0  1  3  0   2  0  0  0  0  0  4  0  0  1  0  1
 0  0  0  0  5  0  0  1   1  0  1  0  2  2  4  1  1  5  3  0
 0  0  0  0  1  0  1  1   0  6  1  1  0  0  0  1  0  1  0  2
 2  0  1  1  0  2  0  0   1  0  0  1  2  0  5  1  0  2  1  1
 0  0  0  0  3  0  0  2   5  2  0  2  0  3  1  4  1  0  2  2
 1  1  0  0  0  0  2  0   0  1  0  3  1  2  4  2  7  2  2  0
 4  4  3  1  0  0  0  0   0  3  5  2  0  1  0  1  0  1  1  4
 0  0  1  3  0  2  3  1   0  1  1  0  0  2  1  0  1  3  1  1
 3  0  7  1  0  0  2  0   1  1  1  2  0  0  0  1  0  1  1  0
 1  0  1  1  0  0  4  0   1  1  0  0  0  0  2  0  0  1  0  0
 0  0  0  2  1  2  0  3   0  1  0  3  0  0  1  2  0  3  2  0
 0  1  2  2  0  1  2  0   3  0  0  1  0  0  7  0  0  0  1  2
 3  4  1  0  2  1  1  0   2  1  2  5  0  3  0  1  1  2  2  0
 5  0  2  0  0  1  3  3   3  5  1  1  0  2  0  1  1  0  2  3
 0  3  3  0  1  1  1  0   0  1  5  0  1  3  0  0  1  0  0  1
 1  0  2  0  3  0  1  0   0  0  1  0  0  0  0  1  1  4  1  0
 1

In [4]:
# Simulate Dynamic Programming
# T(m, n) = max(T(m-1, n), T(m, n-1)) + s + 1
T = zeros(mat_size, mat_size)
for row_num = 1:mat_size
    for col_num = 1:mat_size
        if (row_num > 1 && col_num > 1)
            # This is really the main case; the rest are just literally the "edges."
            T[row_num, col_num] = (max(T[row_num - 1, col_num], T[row_num, col_num - 1])
                + indep_draws[row_num, col_num] + 1)
        elseif (row_num == 1 && col_num > 1)
            T[row_num, col_num] = (T[row_num, col_num - 1] + indep_draws[row_num, col_num]
                + 1)
        elseif (col_num == 1 && row_num > 1)
            T[row_num, col_num] = (T[row_num - 1, col_num] + indep_draws[row_num, col_num]
                + 1)
        elseif (col_num == 1 && row_num == 1)
            T[row_num, col_num] = indep_draws[row_num, col_num] + 1
        else
            error("Invalid row or column number combination // out of bounds.")
        end
    end
end
T = T .- (T[1,1] - 1) # so that T[1,1] = 1

20×20 Array{Float64,2}:
  1.0   3.0   4.0   6.0   7.0   9.0  …   28.0   29.0   31.0   32.0   34.0
  2.0   4.0   5.0   7.0  13.0  14.0      36.0   38.0   44.0   48.0   49.0
  3.0   5.0   6.0   8.0  15.0  16.0      38.0   39.0   46.0   49.0   52.0
  6.0   7.0   9.0  11.0  16.0  19.0      46.0   47.0   50.0   52.0   54.0
  7.0   8.0  10.0  12.0  20.0  21.0      51.0   53.0   54.0   57.0   60.0
  9.0  11.0  12.0  13.0  21.0  22.0  …   55.0   63.0   66.0   69.0   70.0
 14.0  19.0  23.0  25.0  26.0  27.0      57.0   64.0   68.0   71.0   76.0
 15.0  20.0  25.0  29.0  30.0  33.0      58.0   66.0   72.0   74.0   78.0
 19.0  21.0  33.0  35.0  36.0  37.0      60.0   67.0   74.0   76.0   79.0
 21.0  22.0  35.0  37.0  38.0  39.0      62.0   68.0   76.0   77.0   80.0
 22.0  23.0  36.0  40.0  42.0  45.0  …   66.0   69.0   80.0   83.0   84.0
 23.0  25.0  39.0  43.0  44.0  47.0      72.0   73.0   81.0   85.0   88.0
 27.0  32.0  41.0  44.0  47.0  49.0      77.0   79.0   84.0   88.0   89.0
 33.0  34.0  4

In [5]:
# Visualize corner growth

In [6]:
# analytical limiting shape:
limiting(x, y) = x + y + 2*√(x*y*p);

In [7]:
@manipulate for t in slider(1:maximum(T)+1, value=1)
    # plot which squares are "alive"
    heatmap((T .< t) + (T .== t) .* 0.5, # This makes the new squares a different color
        colorbar = false, aspect_ratio = 1, clims = (0,1),
        xlims = (1, mat_size), ylims = (1, mat_size))
    # plot the theoretical contour
    xs = ys = 1:.1:mat_size
    scaling = t + 2 # Is this scaling correct? Taken from the RandomGrowth notebook in this repo 
    contour!(xs, ys, (x,y)->limiting(x/scaling,y/scaling), colorbar=false, levels = [1-p],
            color=:blue, lw=3)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[nothing], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 119,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/mitchell/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/mitchell/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/mitchell/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/mitchell/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/mitchell/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000001087e5f90, Task (runnable) @0x00000001087e5f90), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1.0\",\"2.0\",\"3.0\",\"4.0\",\"5.0\",\"6.0\",\"7.0\",\"8.0\",\"9.0\",\"10.0\",\"11.0\",\"12.0\",\"13.0\",\"14.0\",\"15.0\",\"16.0\",\"17.0\",\"18.0\",\"19.0\",\"20.0\",\"21.0\",\"22.0\",\"23.0\",\"24.0\",\"25.0\",\"26.0\",\"27.0